In [2]:
# Import necessary libraries
import os
import pandas as pd
from astropy.coordinates import SkyCoord
from astroquery.mast import Catalogs
from astropy import units as u
from tqdm import tqdm
import glob
import requests

In [ ]:
# Step 1.1: Extract SDSS DR18 LRG Samples using CasJobs
# Data listed in dr18_lrg_sample.csv (Sample Size: 1,174,900)
# Criteria : 

# SELECT
  # s.specobjid,
  # s.z AS redshift,                   
  # s.veldisp,
  # p.ra, p.dec, p.u, p.g, p.r, p.i, p.modelMag_r,
  # s.programname, s.plate, s.fiberid, s.mjd into mydb.LRG_full_catalog from SpecObjAll AS s
# JOIN PhotoObjAll AS p ON s.bestobjid = p.objid
# WHERE
  # s.class = 'GALAXY'
  # AND s.z BETWEEN 0.1 AND 0.7
  # AND s.veldisp > 0 AND s.veldisp < 500
  # AND s.programname IN ('boss', 'eboss')
  # AND (p.r - p.i) > 0.5
  # AND (p.g - p.r) > 0.7
  # AND p.modelMag_r BETWEEN 16 AND 21

In [ ]:
# Cross-match SDSS data with the PS1 catalog

BATCH_SIZE = 10000
INPUT_CSV = "sdss_dr18_lrg_sample.csv"
OUTPUT_CSV_TEMPLATE = "sdss_lrg_ps1_matched_batch_{batch_num}.csv"
OUTPUT_FOLDER = "sdss_lrg_queried_objects"

df = pd.read_csv(INPUT_CSV)
n_objects = len(df)

def query_ps1_batch(batch_df):
    g_mag, r_mag, i_mag = [], [], []
    g_depth, r_depth, i_depth = [], [], []
    g_fwhm, r_fwhm, i_fwhm = [], [], []

    for _, row in tqdm(batch_df.iterrows(), total=len(batch_df)):
        try:
            coord = SkyCoord(ra=row['ra'], dec=row['dec'], unit='deg', frame='icrs')
            result = Catalogs.query_region(coord, radius=2.5 * u.arcsec, catalog='PanSTARRS', data_release='dr2')
            if len(result) > 0:
                best = result[0]
                g_mag.append(best.get('gMeanPSFMag'))
                r_mag.append(best.get('rMeanPSFMag'))
                i_mag.append(best.get('iMeanPSFMag'))

                g_depth.append(best.get('gMeanDepth'))
                r_depth.append(best.get('rMeanDepth'))
                i_depth.append(best.get('iMeanDepth'))

                g_fwhm.append(best.get('gFWHM'))
                r_fwhm.append(best.get('rFWHM'))
                i_fwhm.append(best.get('iFWHM'))
            else:
                g_mag.append(None)
                r_mag.append(None)
                i_mag.append(None)
                g_depth.append(None)
                r_depth.append(None)
                i_depth.append(None)
                g_fwhm.append(None)
                r_fwhm.append(None)
                i_fwhm.append(None)
        except Exception:
            g_mag.append(None)
            r_mag.append(None)
            i_mag.append(None)
            g_depth.append(None)
            r_depth.append(None)
            i_depth.append(None)
            g_fwhm.append(None)
            r_fwhm.append(None)
            i_fwhm.append(None)

    batch_df['ps1_g_mag'] = g_mag
    batch_df['ps1_r_mag'] = r_mag
    batch_df['ps1_i_mag'] = i_mag
    batch_df['ps1_g_depth'] = g_depth
    batch_df['ps1_r_depth'] = r_depth
    batch_df['ps1_i_depth'] = i_depth
    batch_df['ps1_g_fwhm'] = g_fwhm
    batch_df['ps1_r_fwhm'] = r_fwhm
    batch_df['ps1_i_fwhm'] = i_fwhm

    # Filter: keep only rows with at least 2 non-null PS1 magnitudes
    mask = (
        batch_df[['ps1_g_mag', 'ps1_r_mag', 'ps1_i_mag']]
        .notnull()
        .sum(axis=1) >= 2
    )
    filtered_df = batch_df[mask].reset_index(drop=True)
    print(f"Filtered from {len(batch_df)} → {len(filtered_df)} rows with ≥ 2 PS1 bands")
    return filtered_df

for i in range(0, n_objects, BATCH_SIZE):
    batch_num = i // BATCH_SIZE + 1
    output_file = os.path.join(OUTPUT_FOLDER, OUTPUT_CSV_TEMPLATE.format(batch_num=batch_num))
    
    if os.path.exists(output_file):
        print(f"Batch {batch_num} already done (found {output_file}), skipping...")
        continue

    batch_df = df.iloc[i:i+BATCH_SIZE].copy()
    print(f"\nProcessing batch {batch_num} ({i} to {i + len(batch_df) - 1})...")
    filtered_df = query_ps1_batch(batch_df)
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    filtered_df.to_csv(output_file, index=False)
    print(f"Saved filtered batch {batch_num} to {output_file}")

print("\nAll batches processed!")

In [4]:
# Calculating color of each object
# Prioritize g-r, then r-i, then g-i, then paste them in each csv file 

INPUT_FOLDER = "sdss_lrg_queried_objects"
CHUNK_SIZE = 10000

def determine_color(row):
    g, r, i = row['ps1_g_mag'], row['ps1_r_mag'], row['ps1_i_mag']
    if pd.notnull(g) and pd.notnull(r):
        row['color_name'] = 'g-r'
        row['color_value'] = g - r
    elif pd.notnull(r) and pd.notnull(i):
        row['color_name'] = 'r-i'
        row['color_value'] = r - i
    elif pd.notnull(g) and pd.notnull(i):
        row['color_name'] = 'g-i'
        row['color_value'] = g - i
    else:
        row['color_name'] = None
        row['color_value'] = None
    return row

for filename in sorted(os.listdir(INPUT_FOLDER)):
    if not filename.endswith(".csv"):
        continue

    input_path = os.path.join(INPUT_FOLDER, filename)
    temp_output_path = os.path.join(INPUT_FOLDER, f"temp_{filename}")
    first_batch = True

    print(f"\nProcessing: {filename}")

    for chunk in pd.read_csv(input_path, chunksize=CHUNK_SIZE):
        # Replace '--' with NaN and convert to float
        for col in ['ps1_g_mag', 'ps1_r_mag', 'ps1_i_mag']:
            if col in chunk.columns:
                chunk[col] = pd.to_numeric(chunk[col], errors='coerce')

        # Apply color computation
        chunk = chunk.apply(determine_color, axis=1)

        # Save to temp file
        chunk.to_csv(
            temp_output_path,
            mode='w' if first_batch else 'a',
            index=False,
            header=first_batch
        )
        first_batch = False
        print(f"  Processed chunk with {len(chunk)} rows")

    # Overwrite original file
    os.replace(temp_output_path, input_path)
    print(f"Updated file saved: {filename}")

print("All CSVs processed with color info added.")


Processing: sdss_lrg_ps1_matched_batch_1.csv
  Processed chunk with 9998 rows
Updated file saved: sdss_lrg_ps1_matched_batch_1.csv

Processing: sdss_lrg_ps1_matched_batch_10.csv
  Processed chunk with 10000 rows
Updated file saved: sdss_lrg_ps1_matched_batch_10.csv

Processing: sdss_lrg_ps1_matched_batch_100.csv
  Processed chunk with 10000 rows
Updated file saved: sdss_lrg_ps1_matched_batch_100.csv

Processing: sdss_lrg_ps1_matched_batch_101.csv
  Processed chunk with 9985 rows
Updated file saved: sdss_lrg_ps1_matched_batch_101.csv

Processing: sdss_lrg_ps1_matched_batch_102.csv
  Processed chunk with 10000 rows
Updated file saved: sdss_lrg_ps1_matched_batch_102.csv

Processing: sdss_lrg_ps1_matched_batch_103.csv
  Processed chunk with 9997 rows
Updated file saved: sdss_lrg_ps1_matched_batch_103.csv

Processing: sdss_lrg_ps1_matched_batch_104.csv
  Processed chunk with 9999 rows
Updated file saved: sdss_lrg_ps1_matched_batch_104.csv

Processing: sdss_lrg_ps1_matched_batch_105.csv
  P

In [ ]:
# Downloading the PS1 cutouts for each object
# FITS images in ps1_cutouts for each valid magnitude 

def ps1_image_list(ra, dec, size=240, filters="gri"):
    """
    Python version of ps1_image_list(): queries the Pan-STARRS image cutouts for available images.

    :param ra: Right Ascension in degrees
    :param dec: Declination in degrees
    :param size: Size in pixels (0.25 arcsec/pix)
    :param filters: Filter string, e.g., "grz"
    :return: DataFrame of available image files
    """
    url = "https://ps1images.stsci.edu/cgi-bin/ps1cutouts"
    params = {
        "pos": f"{ra},{dec}",
        "filter": filters,
        "filetypes": "stack",
        "size": size,
        "format": "json"
    }

    try:
        response = requests.get(url, params=params, timeout=20)
        response.raise_for_status()
        data = response.json()
        return pd.DataFrame(data)
    except Exception as e:
        print(f"Failed to retrieve image list: {e}")
        return pd.DataFrame()

# Example usage for your object:
ra = 212.049238443347
dec = -1.19128574956158
filters = "gri"

df = ps1_image_list(ra, dec, size=240, filters=filters)
print(df.head())

Failed to retrieve image list: Expecting value: line 1 column 1 (char 0)
Empty DataFrame
Columns: []
Index: []
